## 定义数据类

In [1]:
import attrs


@attrs.define
class User:
    username: str
    password: str
    email: str
    is_vip: bool = False

User("100gle", "password", email="email@example.com")

User(username='100gle', password='password', email='email@example.com', is_vip=False)

In [2]:
@attrs.define
class User:
    username: str
    password: str = attrs.field(repr=lambda v: "***")
    email: str
    is_vip: bool = attrs.field(default=False, repr=False)

User("100gle", "password", email="email@example.com")

User(username='100gle', password=***, email='email@example.com')

In [3]:
from datetime import datetime

import attrs


@attrs.define
class User:
    username: str
    email: str
    _password: str = attrs.field(repr=lambda v: "***")
    _is_vip: bool = attrs.field(init=False, default=False, repr=False)
    _recent_login_at: datetime = attrs.field(init=False, default=datetime.now())

In [4]:
User("100gle", email="email@example.com", password="password")

User(username='100gle', email='email@example.com', _password=***, _recent_login_at=datetime.datetime(2022, 11, 19, 10, 44, 48, 848285))

In [5]:
from datetime import datetime

import attr


@attr.s
class User:
    username = attr.ib(type=str)
    email = attr.ib(type=str)
    _password = attr.ib(repr=lambda v: "***")
    _is_vip = attr.ib(default=False, repr=False)
    _recent_login_at = attr.ib(init=False, default=datetime.now())

## 自定义字段校验逻辑

In [6]:
class LimitError(Exception):
    pass


@attrs.define
class User:
    username: str
    email: str = attrs.field()
    _password: str = attrs.field(repr=lambda v: "***")
    _is_vip: bool = attrs.field(default=False, repr=False)
    _recent_login_at: datetime = attrs.field(init=False, default=datetime.now())

    @email.validator
    def _check_email(self, attribute, value: str):
        if not "@" in value:
            raise ValueError("invalid email that doesn't contains an '@' symbol")

        if len(value) >= 15:
            raise LimitError("email out of the max length of 15 chars")

        return value

In [7]:
# User("100gle", email="email#example.com", password="password")  # raise ValueError here!

In [8]:
import attrs

class LimitError(Exception):
    pass

def check_email_validation(instance, attribute, value: str):
    if not "@" in value:
        raise ValueError("invalid email that doesn't contains an '@' symbol")


def check_email_length(instance, attribute, value: str):
    if len(value) >= 15:
        raise LimitError("email out of the max length of 15 chars")

@attrs.define
class User:
    username: str
    email: str = attrs.field(
            validator=[
                check_email_validation,
                check_email_length,
            ]
        )
    _password: str = attrs.field(repr=lambda v: "***")
    _is_vip: bool = attrs.field(default=False, repr=False)
    _recent_login_at: datetime = attrs.field(init=False, default=datetime.now())

    @email.validator
    def _check_email_suffix(self, attribute, value: str):
        if not value.endswith("com"):
            raise ValueError("invalid domain")


In [9]:
# User("100gle", email="email#example.com", password="password")  # raise ValueError here!

## 自定义转换器

In [10]:
@attrs.define
class C:
    x: int = attrs.field(converter=int)

In [11]:
o = C("1")
o.x


1

In [12]:
type(o.x)

int

In [13]:
from datetime import datetime

import attrs


class LimitError(Exception):
    pass


def fix_invalid_email(value: str) -> str:
    if not "@" in value and "#" in value:
        return value.replace("#", "@")
    return value

@attrs.define
class User:
    username: str
    email: str = attrs.field(converter=fix_invalid_email)
    _password: str = attrs.field(repr=lambda v: "***")
    _is_vip: bool = attrs.field(default=False, repr=False)
    _recent_login_at: datetime = attrs.field(init=False, default=datetime.now())

    @email.validator
    def _check_email_suffix(self, attribute, value: str):
        if not value.endswith("com"):
            raise ValueError("invalid domain")

In [14]:
User("100gle", email="email#example.com", password="password")

User(username='100gle', email='email@example.com', _password=***, _recent_login_at=datetime.datetime(2022, 11, 19, 10, 44, 49, 229669))

In [15]:
# User("100gle", email="email@example.net", password="password")  # raise ValueError here!

## 导出数据类

In [16]:
user = User("100gle", email="email#example.com", password="password")
attrs.asdict(user)

{'username': '100gle',
 'email': 'email@example.com',
 '_password': 'password',
 '_is_vip': False,
 '_recent_login_at': datetime.datetime(2022, 11, 19, 10, 44, 49, 229669)}

In [17]:
attrs.asdict(
    user,
    filter=lambda attr, value: not attr.name.startswith("_"),
)

{'username': '100gle', 'email': 'email@example.com'}

In [18]:
from attrs import fields, filters

attrs.asdict(
    user,
    filter=filters.include(
        fields(User).username,
        fields(User).email,
    ),
)

{'username': '100gle', 'email': 'email@example.com'}

In [19]:
attrs.asdict(
    user,
    filter=filters.exclude(
        fields(User)._password,
        fields(User)._is_vip,
        datetime,
    ),
)

{'username': '100gle', 'email': 'email@example.com'}